### Load the .env file

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

### Import libraries

In [3]:
import os

### Create Openai client

In [4]:
from openai import OpenAI

In [5]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

### Create an assistant

In [6]:
math_tutor = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    name="Math Tutor",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
)

In [7]:
print(math_tutor.model_dump_json(indent=2))

{
  "id": "asst_QC905KgvpwBIFbwd3jmmeyIO",
  "created_at": 1700199008,
  "description": null,
  "file_ids": [],
  "instructions": "You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
  "metadata": {},
  "model": "gpt-4",
  "name": "Math Tutor",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


### Create a new thread for the conversation

In [8]:
thread = client.beta.threads.create()

In [9]:
print(thread.model_dump_json(indent=2))

{
  "id": "thread_XeeN6GpwvyfUN9OH1574vpuD",
  "created_at": 1700199010,
  "metadata": {},
  "object": "thread"
}


### Add message to the thread

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [11]:
print(message.model_dump_json(indent=2))

{
  "id": "msg_ONswLyqsmnLT2uVG6F1el3uV",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "I need to solve the equation `3x + 11 = 14`. Can you help me?"
      },
      "type": "text"
    }
  ],
  "created_at": 1700199020,
  "file_ids": [],
  "metadata": {},
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_XeeN6GpwvyfUN9OH1574vpuD"
}


### Run the query

In [12]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=math_tutor.id,
  instructions="Please address the user as Raj."
)

In [13]:
print(run.model_dump_json(indent=2))

{
  "id": "run_IkXvWWddfgzZglTUMeqLk0oH",
  "assistant_id": "asst_QC905KgvpwBIFbwd3jmmeyIO",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1700199023,
  "expires_at": 1700199623,
  "failed_at": null,
  "file_ids": [],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4",
  "object": "thread.run",
  "required_action": null,
  "started_at": null,
  "status": "queued",
  "thread_id": "thread_XeeN6GpwvyfUN9OH1574vpuD",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


### Check run status

In [22]:
retribed_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [23]:
print(retribed_run.model_dump_json(indent=2))

{
  "id": "run_IkXvWWddfgzZglTUMeqLk0oH",
  "assistant_id": "asst_QC905KgvpwBIFbwd3jmmeyIO",
  "cancelled_at": null,
  "completed_at": 1700199040,
  "created_at": 1700199023,
  "expires_at": null,
  "failed_at": null,
  "file_ids": [],
  "instructions": "Please address the user as Raj.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4",
  "object": "thread.run",
  "required_action": null,
  "started_at": 1700199023,
  "status": "completed",
  "thread_id": "thread_XeeN6GpwvyfUN9OH1574vpuD",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


### Show the response

In [24]:
thread_messages = client.beta.threads.messages.list(thread.id)

In [25]:
user = ''
assistant = ''

for tm in reversed(thread_messages.data):
    content = tm.content[0].text.value
    if tm.role == 'user':
        user += content
    if tm.role == 'assistant':
        assistant += content

In [26]:
print(user)

I need to solve the equation `3x + 11 = 14`. Can you help me?


In [27]:
print(assistant)

Sure, Raj. To solve the equation `3x + 11 = 14` for `x`, we can proceed in the following steps:

- First, subtract `11` from both sides of the equation.
- Then, divide by `3`.

Let me calculate that for you.The solution to the equation `3x + 11 = 14` is `x = 1`. This answers the question, Raj. Is there anything else that you would like to know?


### Delete the assistant

In [28]:
client.beta.assistants.delete('asst_QC905KgvpwBIFbwd3jmmeyIO')

AssistantDeleted(id='asst_QC905KgvpwBIFbwd3jmmeyIO', deleted=True, object='assistant.deleted')